# <font color='blue'>Prices Optimization and Product Mix</font>

In [1]:
# Python Language Version
from platform import python_version
print('Python Language Version Used in This Jupyter Notebook:', python_version())

Python Language Version Used in This Jupyter Notebook: 3.7.16


## Defining the Problem

Smart Moon Tech assembles and tests two smartphone models, Moon1 and Moon2. For the next month, the company wants to decide how many units of each model it will assemble and then test.

No smartphone is in stock since the previous month and as these models will be replaced after this month, the company doesn't want to keep any stock for the following month.

They believe that the most they can sell this month is 600 Moon1 units and 1200 Moon2 units.

Each Moon1 model is sold for 300 and each Moon2 model for 450. The component cost for a Moon1 is 150 and for a Moon2 it is 225.

Labor is required for assembly and testing. There are a maximum of 10,000 hours of assembly and 3,000 hours of testing available. Each working hour for assembly costs 11 and each working hour for testing costs 15. Each Moon1 requires five hours for assembly and one hour for testing. Each Moon2 requires six hours for assembly and two hours for testing.

Smart Moon Tech wants to know how many units of each model it should produce (build and test) to maximize its net profit, but it can't use more man hours than it has available and it doesn't want to produce more than it can sell.

We will optimize prices and mix of Smart Moon Tech products.

## Loading Packages

In [2]:
# Imports
from pulp import *

## Creating the Mathematical Model for the Optimization

### Parameters

- Ai = Maximum number of type i smartphones to sell this month, where i belongs to the set {Moon1, Moon2}
- Bi = Sale price of smartphones model type i, where i belongs to the set {Moon1, Moon2}
- Ci = Cost price of component parts for type i smartphones, where i belongs to the set {Moon1, Moon2}
- Di = Assembly labor cost per hour of type i smartphones, where i belongs to the set {Moon1, Moon2}
- Ei = Cost of test labor per hour of smartphones model type i, where i belongs to the set {Moon1, Moon2}
- F = Maximum number of hours of assembly work
- G = Maximum number of hours of test work
- Hf,i = Hours of assembly required to build each model of smartphone type i, where i belongs to the set {Moon1, Moon2}
- Hg,i = Hours of testing required to test each smartphone model type i, where i belongs to the set {Moon1, Moon2}

### Decision Variable

- Xi = Type i smartphone numbers to be produced this month, where i belongs to the set {Moon1, Moon2}

### Restrictions

- The number of type i smartphones to be produced cannot be negative, that is, Xi >= 0, where i belongs to the set {Moon1, Moon2}.


- The total number of assembly hours cannot be greater than the maximum number of assembly labor hours available.


- The total number of test hours cannot be greater than the maximum test manpower hours available.


- The number of type i smartphones to be produced cannot be greater than the maximum number of type i smartphones to be sold in this month, where i belongs to the set {Moon1, Moon2}.

## Mathematical Model Implementing 

### Parameters Organizing

In [3]:
# Maximum number of smartphones to sell this month
A_Moon1 = 600
A_Moon2 = 1200

In [4]:
# Smartphone selling price
B_Moon1 = 300 
B_Moon2 = 450 

In [5]:
# Cost price of component parts for smartphones
C_Moon1 = 150
C_Moon2 = 225

In [6]:
# Assembly labor cost per hour of smartphones
D_Moon1 = 11
D_Moon2 = 11

In [7]:
# Testing labor cost per hour of smartphones
E_Moon1 = 15 
E_Moon2 = 15 

In [8]:
# Maximum number of hours of assembly work
F = 10000 

In [9]:
# Maximum number of hours of test work
G = 3000 

In [10]:
# Hours of assembly required to build each smartphone model
Hfi_Moon1 = 5
Hfi_Moon2 = 6

In [11]:
# Hours of testing required to test each smartphone model
Hgi_Moon1 = 1 
Hgi_Moon2 = 2

### Creating the Variable for the Optimization Problem

In [12]:
help(LpProblem)

Help on class LpProblem in module pulp.pulp:

class LpProblem(builtins.object)
 |  LpProblem(name='NoName', sense=1)
 |  
 |  An LP Problem
 |  
 |  Methods defined here:
 |  
 |  __getstate__(self)
 |  
 |  __iadd__(self, other)
 |  
 |  __init__(self, name='NoName', sense=1)
 |      Creates an LP Problem
 |      
 |      This function creates a new LP Problem  with the specified associated parameters
 |      
 |      :param name: name of the problem used in the output .lp file
 |      :param sense: of the LP problem objective.                  Either :data:`~pulp.const.LpMinimize` (default)                 or :data:`~pulp.const.LpMaximize`.
 |      :return: An LP Problem
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __setstate__(self, state)
 |  
 |  add(self, constraint, name=None)
 |  
 |  addConstraint(self, constraint, name=None)
 |  
 |  addVariable(self, variable)
 |      Adds a variable to the problem before a constraint is added
 |      
 |      @param variable:

In [13]:
# Variable for the problem
problem = LpProblem("ProductMix", LpMaximize)

In [14]:
# Visualize
problem

ProductMix:
MAXIMIZE
None
VARIABLES

### Defining the Decision Variable for Each Smartphone Model

In [15]:
help(LpVariable)

Help on class LpVariable in module pulp.pulp:

class LpVariable(LpElement)
 |  LpVariable(name, lowBound=None, upBound=None, cat='Continuous', e=None)
 |  
 |  This class models an LP Variable with the specified associated parameters
 |  
 |  :param name: The name of the variable used in the output .lp file
 |  :param lowBound: The lower bound on this variable's range.
 |      Default is negative infinity
 |  :param upBound: The upper bound on this variable's range.
 |      Default is positive infinity
 |  :param cat: The category this variable is in, Integer, Binary or
 |      Continuous(default)
 |  :param e: Used for column based modelling: relates to the variable's
 |      existence in the objective function and constraints
 |  
 |  Method resolution order:
 |      LpVariable
 |      LpElement
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, lowBound=None, upBound=None, cat='Continuous', e=None)
 |      Initialize self.  See help(type(self)) for 

In [16]:
# Define the variables
x_Moon1 = LpVariable("Moon1 Units", 0, None, LpInteger)
x_Moon2 = LpVariable("Moon2 Units", 0, None, LpInteger)

In [17]:
# Print
print(x_Moon1)
print(x_Moon2)

Moon1_Units
Moon2_Units


### Implementing the Objective Function

In [18]:
# Revenue
revenue = (x_Moon1 * B_Moon1) + (x_Moon2 * B_Moon2)
revenue

300*Moon1_Units + 450*Moon2_Units + 0

In [19]:
# Assembly Cost
assembly_cost = (x_Moon1 * Hfi_Moon1 * D_Moon1) + (x_Moon2 * Hfi_Moon2 * D_Moon2)
assembly_cost

55*Moon1_Units + 66*Moon2_Units + 0

In [20]:
# Test Cost
test_cost = (x_Moon1 * Hgi_Moon1 * E_Moon1) + (x_Moon2 * Hgi_Moon2 * E_Moon2)
test_cost

15*Moon1_Units + 30*Moon2_Units + 0

In [21]:
# Components Cost
components_cost = (x_Moon1 * C_Moon1) + (x_Moon2 * C_Moon2) 
components_cost

150*Moon1_Units + 225*Moon2_Units + 0

In [22]:
# Profit = Revenue - Assembly Cost - Test Cost - Component Cost
problem += revenue - assembly_cost - test_cost - components_cost
problem

ProductMix:
MAXIMIZE
80*Moon1_Units + 129*Moon2_Units + 0
VARIABLES
0 <= Moon1_Units Integer
0 <= Moon2_Units Integer

### Restrictions

In [23]:
# Maximum Number of Assembly Hours
problem += (x_Moon1 * Hfi_Moon1) + (x_Moon2 * Hfi_Moon2) <= F,"Maximum Number of Assembly Hours"

In [24]:
# Maximum Number of Test Hours
problem += (x_Moon1 * Hgi_Moon1) + (x_Moon2 * Hgi_Moon2) <= G,"Maximum Number of Test Hours"

In [25]:
# Production less than or equal to demand for the Moon1 model
problem += x_Moon1 <= A_Moon1,"Production less than or equal to demand for the Moon1 model"

In [26]:
# Production less than or equal to demand for the Moon2 model
problem += x_Moon2 <= A_Moon2,"Production less than or equal to demand for the Moon2 model"

In [27]:
# Final problem
problem

ProductMix:
MAXIMIZE
80*Moon1_Units + 129*Moon2_Units + 0
SUBJECT TO
Maximum_Number_of_Assembly_Hours: 5 Moon1_Units + 6 Moon2_Units <= 10000

Maximum_Number_of_Test_Hours: Moon1_Units + 2 Moon2_Units <= 3000

Production_less_than_or_equal_to_demand_for_the_Moon1_model: Moon1_Units
 <= 600

Production_less_than_or_equal_to_demand_for_the_Moon2_model: Moon2_Units
 <= 1200

VARIABLES
0 <= Moon1_Units Integer
0 <= Moon2_Units Integer

### Solving the Optimization problem

In [28]:
# Optimization
problem.solve()

1

In [29]:
# Maximized Profit
print("Maximized Profit:", value(problem.objective))

Maximized Profit: 199600.0


In [30]:
# Number of Moon1 Model Units to be Produced
print("Number of Moon1 Model Units to be Produced:", problem.variables()[0].varValue)

Number of Moon1 Model Units to be Produced: 560.0


In [31]:
# Number of Moon2 Model Units to be Produced
print("Number of Moon2 Model Units to be Produced:", problem.variables()[1].varValue)

Number of Moon2 Model Units to be Produced: 1200.0


##  Conclusion

The company Moon Smart Tech must produce 560 Moon1 units and 1200 Moon2 units to reach the maximum profit of 199,600.

# End